In [77]:
import os
from datetime import datetime, timedelta
from pathlib import Path
from dateutil import rrule
from prometheus_api_client import PrometheusConnect
from prometheus_pandas import query
date_format_str = '%Y-%m-%dT%H:%M:%SZ'
prom = PrometheusConnect(url="http://127.0.0.1:9090/", disable_ssl=True)

p = query.Prometheus("http://127.0.0.1:9090/")
res = p.query_range('rate(node_disk_info)', cd[0], cd[1], "1s")

p.query()

RuntimeError: bad_data: 1:6: parse error: expected type range vector in call to function "rate", got instant vector

In [ ]:
import os
from datetime import datetime, timedelta
from pathlib import Path
from dateutil import rrule
from prometheus_api_client import PrometheusConnect
from prometheus_pandas import query
date_format_str = '%Y-%m-%dT%H:%M:%SZ'
def chunk_datetime(start_time_str: str, end_time_str: str, interval: int = 3):

    def hours_aligned(start, end, interval, inc=True):
        if inc: yield start
        rule = rrule.rrule(rrule.HOURLY, interval=interval, byminute=0, bysecond=0, dtstart=start)
        for x in rule.between(start, end, inc=inc):
            yield x
        if inc: yield end

    start_time = datetime.strptime(start_time_str, date_format_str)
    end_time = datetime.strptime(end_time_str, date_format_str)
    time_list = list(hours_aligned(start_time, end_time, interval))

    result = []
    for i in range(len(time_list) - 1):
        if i == 0:
            result.append([time_list[i], time_list[i + 1]])
            continue
        result.append([time_list[i] + timedelta(seconds=1), time_list[i + 1]])

    return result

prom = PrometheusConnect(url="http://127.0.0.1:9090/", disable_ssl=True)

# prometheus unique save dir
prom_save_dir = "cpu_rate"
# final save dir
save_dir = os.path.join("data", prom_save_dir)
Path(save_dir).mkdir(parents=True, exist_ok=True)

chunked_datetime = chunk_datetime("2022-02-18T11:30:00Z", "2022-02-25T11:30:00Z")


for i, cd in enumerate(chunked_datetime):
    p = query.Prometheus("http://127.0.0.1:9090/")
    res = p.query_range('rate(node_disk_info)', cd[0], cd[1], "1s")
    # res = p.query_range('sum(1-rate(node_cpu_seconds_total{mode="idle"}[1m]))', cd[0], cd[1], "1s")
    save_path = os.path.join(save_dir, ('fail_test' + ".csv"))
    if i == 0:
        res.to_csv(save_path)
    else:
        res.to_csv(save_path, mode='a', header=False)

In [ ]:
import pandas as pd
df = pd.read_csv('data/cpu_rate/rate(node_cpu_seconds_total{mode="idle"}[1m]).csv')
column_names = list(df.columns)
df.set_index(column_names[0],inplace=True)
df.index = pd.to_datetime(df.index, infer_datetime_format=True)
# df['Total']= df[column_names[:-1]].sum(axis=1)

In [ ]:
df = df["2022-02-18 11:30:00":]

In [ ]:
df = df.resample("1min").mean()
df.plot()

In [ ]:
df.index.name = "Time"

In [ ]:
df

In [ ]:
df = df.rename(columns={'{}': 'cpu_rate'})

In [ ]:
df.to_csv("data/cpu_rate/cpu_rate_test.csv")

In [78]:
import time
print(time.ctime())


Mon Feb 28 12:29:53 2022
